# Travel Agentic Application

This notebook implements the agentic travel application presented in the *Agentic Applications* document on the [Responsible AI Github repo](https://github.com/microsoft/responsible-ai-workshop).


To avoid any problems when using this notebook, please refer to the instructions and technical documentation in the associated document.


If it's your first time using a notebook or a notebook associeted to our repo, please read the **Prerequisites Guides**.

You will also need to read the note:
- [getting-started-with-azure](https://github.com/microsoft/responsible-ai-workshop/blob/main/perequisites/getting-started-with-azure.md) 
- [creation-in-Azure-and-using-it-in-python](https://github.com/microsoft/responsible-ai-workshop/blob/main/perequisites/creation-in-Azure-and-using-it-in-python.md)
- [installing-microsoft-visual-C++](https://github.com/microsoft/responsible-ai-workshop/blob/main/perequisites/installing-microsoft-visual-C%2B%2B.md)

We recommend also to instal the Markdown Extension in VS Code to preview the markdown files (right click on the .md file to access to the preview option)

## Install Packages

In [ ]:
#Install packages
%pip install pyautogen
%pip install openai
%pip install chroma
%pip install python-dotenv
%pip install markdownify
%pip install apify_client
%pip install duckduckgo-search

## Import

In [2]:
import os
from autogen import GroupChat, GroupChatManager, UserProxyAgent, AssistantAgent, register_function, ConversableAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb
from dotenv import load_dotenv
from duckduckgo_search import DDGS
from typing import Annotated
import datetime

## Initialization

For this step, if you haven't already done so, you first need to follow the instructions from the document: [creation-in-Azure-and-using-it-in-python](https://github.com/microsoft/responsible-ai-workshop/blob/main/perequisites/creation-in-Azure-and-using-it-in-python.md).

Then put the following parameters:
- model: the name you gave to your model
- api_version: a version (if it's possible, the latest)  from this [link](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference).

In [2]:
load_dotenv()

llm_config_dict = {"config_list": [{"model": "gpt-4o",                # way more quick with gpt-4o-mini but can't use multiple tools                            
                                    "api_type": "azure",
                                    "api_key": os.environ["AZURE_OPENAI_API_KEY"],
                                    "api_version": "2024-06-01", 
                                    "base_url": os.environ["AZURE_OPENAI_API_BASE"]}],
                    "cache_seed": 42, 
                    "temperature": 0}

# Simple Travel Agentic Application

## Structure of the Application

As a first step, we can create a simple travel agent application, made up of several agents who will have to deal with the problems at hand.


Here, we're going to use **Conversational Agents** without any external tools (we'll add more in the advanced version of the application that we'll create in the next section).


The agents will communicate in the form of a peer chain. In other words, they will communicate two by two and pass on their discussion to the next group, which will try to go further or take other aspects into account.



<img src="./images/Dessin (5).png" alt="GroupChat Flow" width="1000" style="display: block; margin-left: auto; margin-right: auto;">


Our application consists of five agents:
- an agent who selects the destination
- an agent for transport
- an agent for accommodation
- an agent for activities
- a reporting agent who discusses with each agent


## Creation of Agents

In [3]:
report_agent = ConversableAgent(
    name="Report_Agent",
    system_message="You are responsible for creating a report by extracting insights from the chat history.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# Destination Agent
destination_agent = ConversableAgent(
    name="Destination_Agent",
    system_message="Help choose the destination based on your expertise."
                    "Provide at most 3 options with a brief description of each.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# Transport Agent
transport_agent = ConversableAgent(
    name="Transport_Agent",
    system_message="Find the best transport options to get to the chosen destination regarding time, cost, and environmental impact."
                    "In the end, select the best option and provide a brief description.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# Accommodation Agent
accommodation_agent = ConversableAgent(
    name="Accommodation_Agent",
    system_message="Recommend accommodation options that fit the traveler's needs and budget (hotels, Airbnb, hostels)",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# Activities Agent
activities_agent = ConversableAgent(
    name="Activities_Agent",
    system_message="Suggest activities and attractions to do at the destination, based on the traveler’s interests.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

## Launch the discussion

In [10]:
# Here we start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.

chat_results = report_agent.initiate_chats(
    [
        {
            "recipient": destination_agent,
            "message": "What are the best places for a sunny vacation in December, knowing that I'm in Paris?",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": transport_agent,
            "message": "What are the best transport options to get to these destinations whitout a big footprint?",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": accommodation_agent,
            "message": "What are the top-rated hotels in the selected destination?",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": activities_agent,
            "message": "What are the best activities in the selected destination?",
            "max_turns": 1,
            "summary_method": "last_msg",
        }
    ]
)



********************************************************************************
Starting a new chat....

********************************************************************************
Report_Agent (to Destination_Agent):

What are the best places for a sunny vacation in December, knowing that I'm in Paris?

--------------------------------------------------------------------------------
Destination_Agent (to Report_Agent):

Sure! Here are three excellent sunny vacation destinations for December, considering you're starting from Paris:

1. **Canary Islands, Spain**:
   - **Description**: Located off the northwest coast of Africa, the Canary Islands offer warm temperatures and plenty of sunshine in December. With beautiful beaches, volcanic landscapes, and charming towns, it's a great destination for both relaxation and adventure.
   - **Travel Time**: Approximately 4-5 hours by direct flight from Paris.

2. **Dubai, United Arab Emirates**:
   - **Description**: Dubai boasts luxuriou

In [9]:
print("First Chat Summary: ", chat_results[0].summary)
print("\n******************************************************************************************\n")
print("Second Chat Summary: ", chat_results[1].summary)
print("\n******************************************************************************************\n")
print("Third Chat Summary: ", chat_results[2].summary)
print("\n******************************************************************************************\n")
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  Sure, here are three diverse travel destinations, each offering unique experiences:

1. **Kyoto, Japan**:
   - **Description**: Known for its classical Buddhist temples, as well as gardens, imperial palaces, Shinto shrines, and traditional wooden houses. Kyoto is also famous for its formal traditions such as kaiseki dining and geisha female entertainers.
   - **Highlights**: Fushimi Inari Shrine, Kinkaku-ji (Golden Pavilion), Arashiyama Bamboo Grove, and the Gion district.

2. **Santorini, Greece**:
   - **Description**: A stunning island in the Aegean Sea, known for its white-washed buildings with blue domes, dramatic views, and beautiful sunsets. It’s a perfect destination for romance, relaxation, and exploring ancient ruins.
   - **Highlights**: Oia village, Akrotiri archaeological site, Red Beach, and the island’s wineries.

3. **Banff National Park, Canada**:
   - **Description**: Located in the Canadian Rockies, Banff is renowned for its stunning mountain lan

# Advanced Travel Agentic Application

## Structure of our Travel Agentic Application

This part is a reminder of the description of the application in the associated text document. 


You can skip it and move on to the next part if you wish to continue with the coding steps.

Our application has the following structure:

<img src="images/Dessin (3).png" alt="Application Structure" width="1000" style="display: block; margin-left: auto; margin-right: auto;">

This structure is made up of:
- **User** which results in the generation of the original prompt and potential exchanges with the UserProxy agent if the application needs the user's itervention
- **UserProxyAgent**, which is a conversational agent that handles interaction between the application and the user
- **Planner Agent**, which plans the tasks to be carried out according to the problem to be solved. It breaks down the problem into sub-tasks and assigns them to an agent
- **Personnal Advisor agent** with access to user files. It responds to a problem by using a document search agent (with RAG technology) to synthesise information based on the request
- **WebSearchAgent** searches on the Internet for additional information and summarises it according to the problem
- **Report Creator**, which summarises the discussion between the various agents into a markdown file that addresses the initial problem. It also takes care of ending the conversation.

This is the call system or our application:

<img src="images/Orga.png" alt="Application Structure" width="1000" style="display: block; margin-left: auto; margin-right: auto;">


Here, the **User** sends a prompt to the **UserProxyAgent**, which forwards it to the **GroupChat Manger**. 

The GroupChat Manger will then call the most appropriate agent according to the request. 

This agent will respond to the request, create new ones, potentially use tools and transmit all the information to the **GroupChat Manager**. 

The **GroupChat Manager** then selects the most relevant agent and sends it the information, and so on until the task is completed and the process is stopped.

## Let's define our agents

The first step is to define our agents by setting their parameters according to their role 


In [15]:
# Parameters for the personnal_advisor
PATH_TO_NOTEBOOK = os.environ["PATH_TO_NOTEBOOK"]   # Modify the path in the .env file
DOC_PATH = [PATH_TO_NOTEBOOK+"vacations"]           # DO NOT MODIFY 

In [41]:
def terminate_conversation(x):
    if x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"):
        return True

user_proxy = UserProxyAgent(
    name="User_proxy",
    system_message="""A human admin. """, #Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin. Execute the plan by making sure each agent does their part and help in the report creation.
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
    is_termination_msg=terminate_conversation,
    max_consecutive_auto_reply=3
)

planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin.
                    The plan may involve input from PersonalAdvisor and TravelWebSearchAgent.
                    Explain the plan first. Be clear which step is performed by an Personnal Advisor, 
                    and which step is performed by a Travel WebSearch Agent.
                    For save queries, use the reportagent.
                    """,
    llm_config=llm_config_dict,
    description="Suggest a plan involving input from other agents.",
    is_termination_msg=terminate_conversation
)

reportagent = AssistantAgent(
    name="Report_Agent",
    system_message="""
                    You are a reporter who have to extract insights from the other agents
                    to answer the User_proxy's questions.
                    The report should be as short as possible, but include all necessary information.
                    Write the report in well formatted markdown format, use a tool to save it.
                    Once you've done everything, write TERMINATE in the end of your message.""",
    llm_config=llm_config_dict,
    description="Reporter which answer the original question using past agents conversation."
)

personnal_advisor = AssistantAgent(
    name="PersonalAdvisor",
    system_message="""
                    You're a personal travel adviser who has access to files on my previous trips. 
                    You are responsible for providing information about the previous trips to help in planning the new trip.
                    """,    
    llm_config=llm_config_dict,
    description="Personal Advisor, expert in travel planning and  have access to files on previous trips."
)

rag_agent = RetrieveUserProxyAgent(
    name="RAG_Agent",
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": DOC_PATH,
        "chunk_token_size": 1000,
        "model": llm_config_dict["config_list"][0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "get_or_create": True, 
    },
    code_execution_config={"use_docker": False},
    description="Assistant who has extra content retrieval power for answering questions."
) 

travelWebSearchAgent = AssistantAgent(
    name="TravelWebSearchAgent",
    system_message="""You're a travel planning expert with access to the Internet. You're creative and know how to meet every need.
                    Your objective is to respond to requests from other agents to organise a trip that meets their needs by using 
                    information on Internet and your knowledge.""",
    llm_config=llm_config_dict,
    description="Expert travel planning using Internet"
)

## Empower them with Tools

In this section we will create and empower our agents with tools such as *WebSearch*, *Report Creation* and *Use of Documents* (with RAG technology).

In [42]:
# ---------------------------------------- CREATION of functions ----------------------------------------
def save_report(report: str) -> str:
    try:
        file_name = "report_"+str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S"))+ ".md"
        with open(file_name, "w") as f:
            f.write(report)
        return "Report saved successfully."
    except Exception as e:
        print(e)
        return "Failed to save the report."
    
def browse_web(query: str) -> str:
    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.text(query, max_results=1)]
            return results if results else "Not Found"
    except Exception as e:
        print(e)

def retrieve_content(
        message: Annotated[
            str,
            "Refined message which keeps the original meaning and can be used to retrieve content for question answering.",
        ],
        n_results: Annotated[int, "number of results"] = 3,
    ) -> str:
    try:
        rag_agent.n_results = n_results  # Set the number of results to be retrieved.
        _context = {"problem": message, "n_results": n_results}
        ret_msg = rag_agent.message_generator(rag_agent, None, _context)
        return ret_msg or message
    except Exception as e:
        print(e)
        return "Failed to retrieve content."

# ---------------------------------------- TRANSLATION into Tools ----------------------------------------
def web_search_tool (query: Annotated [str, 'Query string containing information that you want to search using the internet']) -> str:
    result=browse_web(query)
    return result

def save_report_tool(report: Annotated[str, 'Report in markdown format']) -> str:
    result=save_report(report)
    return result

def retrieve_content_tool(message: Annotated[str, 'Refined message which keeps the original meaning and can be used to retrieve content for question answering.'],
                     n_results: Annotated[int, 'number of results'] = 3) -> str:
    result=retrieve_content(message, n_results)
    return result

# ---------------------------------------- Empower Agents ----------------------------------------
register_function(
    web_search_tool,
    caller=travelWebSearchAgent,
    executor=travelWebSearchAgent,
    description="Web Browser Tool to search the internet for information.",
)

register_function(
    save_report_tool,
    caller=reportagent,
    executor=reportagent,
    description="Tool to save a report in markdown format.",
)

register_function(
    retrieve_content_tool,
    caller=personnal_advisor,
    executor=personnal_advisor,
    description="Retrieve content for question answering.",
)

## Create an agent group 

We are now going to organise our agents into a group with an associated manager.

In [43]:
group_chat = GroupChat(
    agents=[user_proxy, planner, personnal_advisor, travelWebSearchAgent, reportagent],
    messages=[],
    max_round=20,
    send_introductions=True,
    speaker_selection_method="auto"
)

# Group Chat Manager
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config_dict
)

## Let's try our application!

### Try the RAG option

In [37]:
# Initiate the Conversation
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="How much does my travels cost me ?",    
)
# Print the Discussion Summary Report
print(chat_result.summary)

User_proxy (to chat_manager):

How much does my travels cost me ?

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

To determine the cost of your travels, we need to gather information on your past travel expenses. Here's a plan to achieve this:

### Plan

1. **PersonalAdvisor**: Retrieve records of past trips and their associated costs from your files.
2. **TravelWebSearchAgent**: Search for any additional travel expenses that might not be recorded in your files, such as recent trips or online bookings.
3. **Report_Agent**: Compile the information gathered by PersonalAdvisor and TravelWebSearchAgent to provide a comprehensive report on your travel costs.

### Steps

1. **PersonalAdvisor**: 
   - Access your files and retrieve records of past trips, including transportation, accommodation, meals, and other expenses.
   - Summarize the total costs for each trip.

2. **TravelWebSearchAgent**:
   - Search 

2024-10-15 09:54:10,268 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.
2024-10-15 09:54:10,314 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 16 chunks.


Trying to create collection.
VectorDB returns doc_ids:  [['13ed29ba', '1514a05d', 'fecec149', 'c64a3e89', 'f0b96983']]
Adding content of doc 13ed29ba to context.
Adding content of doc 1514a05d to context.
Adding content of doc fecec149 to context.
Adding content of doc c64a3e89 to context.
Adding content of doc f0b96983 to context.
PersonalAdvisor (to chat_manager):

PersonalAdvisor (to chat_manager):

***** Response from calling tool (call_R8buc5JT6B9Nn90v7aRlG4l7) *****
You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: Retrieve records of past trips and their associated costs, including transportation, accommodation, meals, and other expenses.

Context is: # My Luxurious Trip to Malta

**Date of Vacation:** October 10 - October

### Try the search option

In [38]:
# Initiate the Conversation
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="What are the best place to visit in Paris ?",    
)
# Print the Discussion Summary Report
print(chat_result.summary)

User_proxy (to chat_manager):

What are the best place to visit in Paris ?

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

Sure, let's create a plan to find the best places to visit in Paris. Here's the step-by-step plan:

### Step-by-Step Plan

1. **PersonalAdvisor**: 
   - Check previous trip files to see if there are any recommendations or notes on places to visit in Paris.
   - Provide a list of these places if available.

2. **TravelWebSearchAgent**:
   - Conduct an online search to find the most popular and highly recommended places to visit in Paris.
   - Compile a list of these places along with brief descriptions and any relevant details (e.g., opening hours, ticket prices).

3. **Report_Agent**:
   - Combine the information from PersonalAdvisor and TravelWebSearchAgent.
   - Create a comprehensive report listing the best places to visit in Paris.

### Execution

Let's start with **PersonalAd

### Our final use case

In [44]:
# Initiate the Conversation
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="Plan my next vacation, I want a 5 days trip in a new country, similar to my previous trips for a budget of 1000€",    
)
# Print the Discussion Summary Report
print(chat_result.summary) 

User_proxy (to chat_manager):

Plan my next vacation, I want a 5 days trip in a new country, similar to my previous trips for a budget of 1000€

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

Great! Let's break down the plan for your next vacation. We'll involve both the PersonalAdvisor and the TravelWebSearchAgent to ensure we cover all aspects of your trip planning.

### Step-by-Step Plan:

1. **Review Previous Trips (PersonalAdvisor)**
   - **Objective:** Understand the destinations, activities, and preferences from your past trips.
   - **Action:** PersonalAdvisor will access files on your previous trips to gather information on the types of destinations you enjoyed, activities you participated in, and any other relevant preferences.

2. **Destination Suggestions (TravelWebSearchAgent)**
   - **Objective:** Find new countries that match your preferences and fit within your budget.
   - **Action:**

2024-10-15 10:16:54,539 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.


Trying to create collection.


2024-10-15 10:16:54,773 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 16 chunks.


VectorDB returns doc_ids:  [['0de1c1b2', 'f0b96983', 'c64a3e89']]
Adding content of doc 0de1c1b2 to context.
Adding content of doc f0b96983 to context.
Adding content of doc c64a3e89 to context.
PersonalAdvisor (to chat_manager):

PersonalAdvisor (to chat_manager):

***** Response from calling tool (call_aQ8cZTHzDsAvdEtm8fFmQkno) *****
You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: Provide a summary of the user's previous trips, including destinations, activities, and preferences.

Context is: 
### Day 3: South Coast Adventure
- **Activities:**
  - Visit Seljalandsfoss and Skógafoss waterfalls
  - Explore the black sand beach at Reynisfjara
  - Lunch at a local café in Vík
  - Visit the Sólheimajökull glacier
  - Return to Rey